<a href="https://colab.research.google.com/github/RudyMartin/dsai-2024/blob/main/tf_train_rps_cnn_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TENSORFLOW** Rock Paper Scissors with CNN Models
Assumes running T4 GPU Backend on Google Colab

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import os
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import datetime

# Start time
start_time = datetime.datetime.now()

# 1. Mount Google Drive to access the dataset
drive.mount('/content/gdrive')

# 2. Verify and list directories
root_dir = '/content/gdrive/My Drive/'
print(os.listdir(root_dir))

rps_dir = os.path.join(root_dir, 'rps')
if os.path.exists(rps_dir):
    print(f"'rps' directory contents: {os.listdir(rps_dir)}")
else:
    raise FileNotFoundError(f"Directory {rps_dir} does not exist.")

train_dir = os.path.join(rps_dir, 'train')
test_dir = os.path.join(rps_dir, 'test')
model_dir = os.path.join(root_dir, 'model')
print(f"rps directory contents: {os.listdir(rps_dir)}")


# 3. Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 4. Just normalization for testing
test_datagen = ImageDataGenerator(rescale=1./255)

# 5. Load datasets
train_generator = train_datagen.flow_from_directory(
    os.path.join(rps_dir, 'train'),
    target_size=(160, 120),
    batch_size=8,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(rps_dir, 'test'),
    target_size=(160, 120),
    batch_size=8,
    class_mode='categorical'
)

# 6. Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(160, 120, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# 7. Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 8. Define a learning rate schedule (if needed)
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

callback = LearningRateScheduler(scheduler)

# 9. Train the model
history = model.fit(train_generator,
                    epochs=100,  # Increase for real scenario
                    callbacks=[callback],
                    validation_data=test_generator)

# 10. Save the trained model in the Keras format
model.save(os.path.join(rps_dir, 'tf_model.keras'))
print("Model saved to disk in the Keras format.")

# 11. Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# 12. Accuracy per class
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = list(test_generator.class_indices.keys())
# Assuming actual_labels and Y_pred are your actual and predicted labels respectively
# and target_names are the names of the classes
print(classification_report(actual_labels, Y_pred, target_names=target_names, zero_division=0))

# End time
end_time = datetime.datetime.now()
# Calculate duration
duration = end_time - start_time
# Print execution time
print(f"Execution time: {str(duration)}")



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['DSC02677.jpeg', 'Colab Notebooks', 'dscamp_2023', 'dscamp_2022', 'proportions.csv', 'proportions.gsheet', 'activeloop_2023', 'Untitled spreadsheet (1).gsheet', 'Untitled spreadsheet.gsheet', 'YouTube', 'Copy of My Presentation.gslides', 'Introduction .gslides', 'Copy of Introduction .gslides', 'dscamp', 'dscamp_2024_nano', 'ds_camp_2024_trans_hf', 'rps_test', 'rps', 'rps_pics', 'papers']
'rps' directory contents: ['test', 'train', 'models', 'modelsbaseline_1.keras', 'sequential_acc_graph.png', 'sequential_loss_graph.png', 'df_metrics_20240610.csv', 'model.h5', 'model.keras', 'model.pth']
Found 96 images belonging to 3 classes.
Found 18 images belonging to 3 classes.
Epoch 1/100
12/12 [==============================] - 4s 96ms/step - loss: 15.3099 - accuracy: 0.3646 - val_loss: 3.0831 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 2/100
12/12 [=================

Classification Report
              precision    recall  f1-score   support

       paper       0.33      0.50      0.40         6
        rock       0.40      0.33      0.36         6
    scissors       0.50      0.33      0.40         6

    accuracy                           0.39        18
   macro avg       0.41      0.39      0.39        18
weighted avg       0.41      0.39      0.39        18

Execution time: 0:02:21.592218